In [1]:
%matplotlib inline
import pymongo
from pymongo import MongoClient
import matplotlib.pyplot as plt
from pprint import pprint
from collections import defaultdict
from datetime import datetime, timedelta
from functools import partial
db = MongoClient('mongodb-condor.icecube.wisc.edu').condor

In [ ]:
pipeline = [
    {'$match':{'MATCH_EXP_JOBGLIDEIN_ResourceName':{'$exists': True}}},
    {'$group':{'_id':'$MATCH_EXP_JOBGLIDEIN_ResourceName','count':{'$sum':1}}}
]

In [ ]:
db.command('aggregate', 'things', pipeline=pipeline, explain=True)

In [ ]:
groups = db.condor_history.aggregate(pipeline)
pprint(list(groups))

In [2]:
db.condor_history.find_one()

{'BytesRecvd': 1889256.0,
 'BytesSent': 179157.0,
 'ClusterId': 7551238,
 'Cmd': '/scratch/simprod/glidein-gpu2/submit/11477/iceprod_9437.1466225051.328931/iceprod.11477.9437.clsim_1.sh',
 'CommittedTime': 442,
 'CompletionDate': 1466226006,
 'CumulativeSlotTime': 442.0,
 'DiskUsage_RAW': 311105,
 'EnteredCurrentStatus': 1466226006,
 'ExecutableSize_RAW': 3,
 'ExitBySignal': False,
 'ExitCode': 0,
 'ExitStatus': 0,
 'GlobalJobId': 'glidein-simprod.icecube.wisc.edu#7551238.0#1466225079',
 'ImageSize_RAW': 107028204,
 'JobCurrentStartDate': 1466225564,
 'JobStartDate': 1466225564,
 'JobStatus': 4,
 'LastRemoteHost': 'slot1@3326@kepler19.zeuthen.desy.de',
 'MATCH_EXP_JOBGLIDEIN_ResourceName': 'DESY',
 'NumJobStarts': 1,
 'NumShadowStarts': 1,
 'Owner': 'juancarlos',
 'ProcId': 0,
 'QDate': 1466225079,
 'RemoteSysCpu': 39.0,
 'RemoteUserCpu': 213.0,
 'RemoteWallClockTime': 442.0,
 'RequestCpus': 1,
 'RequestDisk': 2000000,
 'RequestMemory': 2000,
 'Requestgpus': 1,
 'ResidentSetSize_RAW': 

In [2]:
def timekey(t):
    return datetime.fromtimestamp(t).strftime('%Y-%m-%d')
def make2list():
    return [0.,0.]                                              

In [75]:
datetime.fromtimestamp(1484050400)

datetime.datetime(2017, 1, 10, 6, 13, 20)

In [ ]:
projection = ['MATCH_EXP_JOBGLIDEIN_ResourceName','EnteredCurrentStatus','CompletionDate',
              'RemoteWallClockTime','CumulativeSlotTime','NunJobStarts']
filter = {'EnteredCurrentStatus':{'$gte':1483250400,'$lt':1484050400}}
# by day
job_restarts = defaultdict(partial(defaultdict,make2list))
cumulative_time = defaultdict(partial(defaultdict,make2list))
success_time = defaultdict(partial(defaultdict,make2list))
for i,row in enumerate(db.condor_history.find(filter=filter,projection=projection)):
    if i > 1000000:
        break
    try:
        if 'MATCH_EXP_JOBGLIDEIN_ResourceName' not in row:
            continue
        site = row['MATCH_EXP_JOBGLIDEIN_ResourceName']
        date = timekey(row['EnteredCurrentStatus'])
        if 'NumJobStarts' in row:
            job_restarts[site][date][0] += int(row['NumJobStarts'])
            job_restarts[site][date][1] += 1
        cumulative_time[site][date][0] += float(row['CumulativeSlotTime'])
        cumulative_time[site][date][1] += 1
        if 'CompletionDate' in row:
            success_time[site][date][0] += float(row['RemoteWallClockTime'])
            success_time[site][date][1] += 1
    except:
        print(row)
        raise
    

In [4]:
pprint(cumulative_time['WIPAC'].keys())

dict_keys(['2017-01-01'])


In [ ]:
db.condor_history.create_index("EnteredCurrentStatus")

In [ ]:
list(db.condor_history.list_indexes())

In [ ]:
db.condor_history.find(filter=filter,projection=projection).explain()